# MATH 210 Introduction to Mathematical Computing

## March 18, 2016

Today's Agenda:

1. Vector Geometry
    * Example: Volume of a Parallelpiped
    * Example: Projections
    * Example: Gram-Schmidt Algorithm
2. Exercises

In [1]:
import numpy as np
import scipy.linalg as la

## 1. Vector Geometry

We can represent vectors as 1-dimensional NumPy arrays and there are NumPy functions for the dot product and (in 3D) cross product of vectors: `np.dot` and `np.cross`. Note however that we need the linear algebra subpackage `scipy.linalg` to compute the norm of a vector.

Let's create two vectors $\mathbf{v}$ and $\mathbf{w}$ in $\mathbb{R}^3$ (with random integer entries contained in $[-5,5]$) and compute their dot product, cross product and norms.

In [2]:
v = np.random.randint(-5,5,3)
v

array([-3,  1, -4])

In [3]:
w = np.random.randint(-5,5,3)
w

array([ 0, -3, -1])

In [4]:
np.dot(v,w)

1

In [5]:
np.cross(v,w)

array([-13,  -3,   9])

In [6]:
la.norm(v)

5.0990195135927845

In [7]:
sum([ entry**2 for entry in v ])**0.5

5.0990195135927845

### Volume of a Parallelpiped

In 3D, the volume of the parallelpiped spanned by vectors $\mathbf{u}$, $\mathbf{v}$ and $\mathbf{w}$ is

$$
V = \left| \, u \cdot ( v \times w ) \, \right|
$$

Define a function which takes three 1-dimensional NumPy arrays of length 3 and returns the volume of the parallelpiped spanned by those vectors.

In [8]:
def vol_para(u,v,w):
    "Compute the volume of the parallelpiped spanned by u,v and w."
    return abs( np.dot( u , np.cross(v,w) ) )

In [9]:
u = np.array([1,0,0])
v = np.array([0,1,0])
w = np.array([0,0,1])
vol_para(u,v,w)

1

In [10]:
u = np.array([1,1,0])
v = np.array([0,1,1])
w = np.array([1,0,1])
vol_para(u,v,w)

2

Note that the function is written in a way that it will accept Python lists of length 3 and compute the corresponding volume.

In [11]:
vol_para([1,1,0],[0,1,1],[1,0,1])

2

### Projections

Define a function which takes two 1-dimensional NumPy arrays (or Python lists) $\mathbf{v}$ and $\mathbf{a}$ (in that order) and returns the projection of $v$ onto $a$:

$$
\mathrm{proj}_{ \mathbf{a} } (\mathbf{v}) = \frac{ \mathbf{v} \cdot \mathbf{a} }{ \mathbf{a} \cdot \mathbf{a} } \ \mathbf{a}
$$

In [12]:
def proj(v,a):
    "Compute the projection of v onto a."
    return np.dot(v,a) / np.dot(a,a) * np.array(a)

In [13]:
proj([1,1],[1,0])

array([ 1.,  0.])

In [14]:
proj([1,1,1],[1,1,0])

array([ 1.,  1.,  0.])

### Gram-Schmidt Algorithm

The Gram-Schmidt algorithm takes a basis $\{ \mathbf{v}_1, \dots , \mathbf{v}_n \}$ of a vector space $V$ and produces an orthonormal basis $\{ \mathbf{e}_1, \dots , \mathbf{e}_n \}$ of $V$:

\begin{align}
\mathbf{x}_1 &= \mathbf{v}_1 \\
\mathbf{x}_2 &= \mathbf{v}_2 - \mathrm{proj}_{\mathbf{x}_1}(\mathbf{v}_2) \\
 & \vdots \\
\mathbf{x}_n &= \mathbf{v}_n - \mathrm{proj}_{\mathbf{x}_1}(\mathbf{v}_n) - \mathrm{proj}_{\mathbf{x}_2}(\mathbf{v}_n) - \cdots - \mathrm{proj}_{\mathbf{x}_{n-1}}(\mathbf{v}_n)
\end{align}

and then normalize $\mathbf{e}_i = \mathbf{x}_i / ||\mathbf{x}_i||$.

Use the Gram-Schmidt algorithm to turn the following basis into a orthonormal basis of $\mathbb{R}^4$:

$$
\mathbf{v}_1 = \begin{bmatrix} 1 & 2 & 3 & 4 \end{bmatrix} \ , \ \
\mathbf{v}_2 = \begin{bmatrix} 1 & 2 & 3 & 2 \end{bmatrix} \ , \ \
\mathbf{v}_3 = \begin{bmatrix} 1 & -2 & 3 & 4 \end{bmatrix} \ , \ \
\mathbf{v}_4 = \begin{bmatrix} 3 & 2 & 3 & 4 \end{bmatrix}
$$

First, let's check that this is a basis of $\mathbb{R^4}$:

In [15]:
V = np.array([[1,2,3,4],[1,2,3,2],[1,-2,3,4],[3,2,3,4]])
print(V)

[[ 1  2  3  4]
 [ 1  2  3  2]
 [ 1 -2  3  4]
 [ 3  2  3  4]]


In [16]:
la.det(V)

-48.0

Therefore, these vectors are linearly independent. As we proceed with the Gram-Schmidt algorithm, let's assemble the vectors $\mathbf{x}_i$ into the rows of a NumPy array $X$ which we intialize as a zero matrix.

In [17]:
X = np.zeros((4,4))
print(X)

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


The first two steps of the Gram-Schmidt process are:

In [18]:
X[0,:] = V[0,:]
X[1,:] = V[1,:] - proj(V[1,:],X[0,:])
print(X)

[[ 1.          2.          3.          4.        ]
 [ 0.26666667  0.53333333  0.8        -0.93333333]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]


Let's check if $\mathbf{x}_1$ and $\mathbf{x}_2$ are perpendicular:

In [19]:
np.dot( X[0,:] , X[1,:] )

1.7763568394002505e-15

Success! Let's proceed with the next two steps:

In [20]:
X[2,:] = V[2,:] - proj(V[2,:],X[0,:]) - proj(V[2,:],X[1,:])
X[3,:] = V[3,:] - proj(V[3,:],X[0,:]) - proj(V[3,:],X[1,:]) - proj(V[3,:],X[2,:])
print(X)

[[  1.00000000e+00   2.00000000e+00   3.00000000e+00   4.00000000e+00]
 [  2.66666667e-01   5.33333333e-01   8.00000000e-01  -9.33333333e-01]
 [  5.71428571e-01  -2.85714286e+00   1.71428571e+00   1.33226763e-15]
 [  1.80000000e+00   3.33066907e-16  -6.00000000e-01   6.99440506e-16]]


Finally, let's normalize the vectors $\mathbf{x}_i$ to obtain an orthonormal basis:

In [21]:
for i in [0,1,2,3]:
    X[i,:] = X[i,:] / la.norm(X[i,:])

print(X)

[[  1.82574186e-01   3.65148372e-01   5.47722558e-01   7.30296743e-01]
 [  1.95180015e-01   3.90360029e-01   5.85540044e-01  -6.83130051e-01]
 [  1.69030851e-01  -8.45154255e-01   5.07092553e-01   3.94090079e-16]
 [  9.48683298e-01   1.75541673e-16  -3.16227766e-01   3.68637514e-16]]


To check our work, we turn the NumPy array $X$ into a NumPy matrix $Q$ and compute $Q Q^T$. If the rest is the identity, then we have succeeded!

In [22]:
Q = np.matrix(X)
I = Q * Q.T
print(I)

[[  1.00000000e+00   1.94289029e-16   1.11022302e-16  -5.55111512e-17]
 [  1.94289029e-16   1.00000000e+00  -3.88578059e-16  -6.38378239e-16]
 [  1.11022302e-16  -3.88578059e-16   1.00000000e+00  -4.99600361e-16]
 [ -5.55111512e-17  -6.38378239e-16  -4.99600361e-16   1.00000000e+00]]


Let's round $I$ to 2 decimal places to see it more clearly.

In [23]:
np.round(I,2)

array([[ 1.,  0.,  0., -0.],
       [ 0.,  1., -0., -0.],
       [ 0., -0.,  1., -0.],
       [-0., -0., -0.,  1.]])

Finally, let's round $Q$ to two decimal places to see our final result more clearly:

In [24]:
np.round(Q,2)

array([[ 0.18,  0.37,  0.55,  0.73],
       [ 0.2 ,  0.39,  0.59, -0.68],
       [ 0.17, -0.85,  0.51,  0.  ],
       [ 0.95,  0.  , -0.32,  0.  ]])

## 2. Exercises

**Exercise.** Compute the dot product, cross product and norms of the vectors $\mathbf{v}$ and $\mathbf{w}$:

(a) $\mathbf{v} = \begin{bmatrix} 1 & -1 & 2 \end{bmatrix}$ and $\mathbf{w} = \begin{bmatrix} 3 & 2 & -1 \end{bmatrix}$

(b) $\mathbf{v} = \begin{bmatrix} 3 & 4 & -1 \end{bmatrix}$ and $\mathbf{w} = \begin{bmatrix} 0 & 1 & 3 \end{bmatrix}$

**Exercise.** Compute the projection of $\mathbf{v} = \begin{bmatrix} 2 & 5 & -1 \end{bmatrix}$ onto the plane spanned by $\mathbf{a} = \begin{bmatrix} 1 & 1 & -1 \end{bmatrix}$ and $\mathbf{b} = \begin{bmatrix} 4 & 0 & 1 \end{bmatrix}$

**Exercise.** Use the Gram-Schmidt algorithm to convert the following basis into an orthonormal basis of $\mathbb{R}^3$:

$$
\mathbf{v}_1 = \begin{bmatrix} 1 & 1 & -1 \end{bmatrix} \ , \ \ 
\mathbf{v}_2 = \begin{bmatrix} 4 & -2 & 1 \end{bmatrix} \ , \ \
\mathbf{v}_3 = \begin{bmatrix} 1 & 1 & 0 \end{bmatrix}
$$